In this segment we will have the trading sentiment using VADER and TEXTBLOB Analysis to create a sentiment tool for the VIX itself to see if its positive or negative today.

In [1]:
# get some libraries that will be useful

import re
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import string
import matplotlib.pyplot as plt
import pandas_datareader as dr
#To remove weekends from dataset
from pandas.tseries.offsets import BDay

# necessary libraries for wordcloud
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image

# the Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
#words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

# function for encoding categories
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
#keras modeling
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, roc_auc_score, accuracy_score

%matplotlib inline

C:\Users\PC\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
Using TensorFlow backend.


Source : 
https://medium.com/@Intellica.AI/vader-ibm-watson-or-textblob-which-is-better-for-unsupervised-sentiment-analysis-db4143a39445

In [3]:
# grab the data #we will first grab the news data set first
combined_news = pd.read_csv("../data/final_dataframe.csv")

In [4]:
combined_news.head()

,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,upordown
0,2008-08-08,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",b'Georgian troops retreat from S. Osettain cap...,...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",0.0
1,2008-08-11,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,b'Welcome To World War IV! Now In High Definit...,...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",0.0
2,2008-08-12,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...","b""The US military was surprised by the timing ...",...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",1.0
3,2008-08-13,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',"b""The commander of a Navy air reconnaissance s...",...,b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,0.0
4,2008-08-14,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refus

In [5]:
#we create a column to combine all 25 headlines.
combined_news['all25'] = combined_news[combined_news.columns[1:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

In [6]:
df = combined_news[['Date','upordown','all25']]

# VADER Sentiment Analysis

Vader is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. VADER uses a combination of A sentiment lexicon is a list of lexical features (e.g., words) which are generally labeled according to their semantic orientation as either positive or negative. VADER not only tells about the Positivity and Negativity score but also tells us about how positive or negative a sentiment is.

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

results = []

for headline in df['all25']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['headline'] = df['all25'] # add headlines for viewing
    results.append(pol_score)

results

[{'neg': 0.214,
  'neu': 0.724,
  'pos': 0.062,
  'compound': -0.9966,
  'headline': 0       b"Georgia 'downs two Russian warplanes' as cou...
  1       b'Why wont America and Nato help us? If they w...
  2       b'Remember that adorable 9-year-old who sang a...
  3       b' U.S. refuses Israel weapons to attack Iran:...
  4       b'All the experts admit that we should legalis...
                                ...                        
  1984    Barclays and RBS shares suspended from trading...
  1985    2,500 Scientists To Australia: If You Want To ...
  1986    Explosion At Airport In Istanbul,Yemeni former...
  1987    Jamaica proposes marijuana dispensers for tour...
  1988    A 117-year-old woman in Mexico City finally re...
  Name: all25, Length: 1989, dtype: object},
 {'neg': 0.135,
  'neu': 0.773,
  'pos': 0.092,
  'compound': -0.9075,
  'headline': 0       b"Georgia 'downs two Russian warplanes' as cou...
  1       b'Why wont America and Nato help us? If they w...
  2      

In [8]:
df['VaderScore'] = pd.DataFrame(results)['compound']

C:\Users\PC\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df.head()

,Date,upordown,all25,VaderScore
0,2008-08-08,0.0,"b""Georgia 'downs two Russian warplanes' as cou...",-0.9966
1,2008-08-11,0.0,b'Why wont America and Nato help us? If they w...,-0.9075
2,2008-08-12,1.0,b'Remember that adorable 9-year-old who sang a...,-0.9739
3,2008-08-13,0.0,b' U.S. refuses Israel weapons to attack Iran:...,-0.9842
4,2008-08-14,0.0,b'All the experts admit that we should legalis...,-0.9774


In [10]:
#To check if there is no difference in closing and opening price.

#create a new column for the difference in the Closing and Opening Price
df['Vadersentimentscore'] = df['VaderScore']
#if closing price is higher then opening price, will assign value 1
df['Vadersentimentscore'] = np.where(df['VaderScore'] > 0,1, df['Vadersentimentscore'])
#if closing price is equal to opening price, will assign value 0
df['Vadersentimentscore'] = np.where(df['VaderScore'] == 0 ,0, df['Vadersentimentscore'])
#if closing price is lower then opening price, will assign value 0
df['Vadersentimentscore'] = np.where(df['VaderScore'] < 0,0, df['Vadersentimentscore'])

C:\Users\PC\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\PC\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\PC\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [12]:
df.head()

,Date,upordown,all25,VaderScore,Vadersentimentscore
0,2008-08-08,0.0,"b""Georgia 'downs two Russian warplanes' as cou...",-0.9966,0.0
1,2008-08-11,0.0,b'Why wont America and Nato help us? If they w...,-0.9075,0.0
2,2008-08-12,1.0,b'Remember that adorable 9-year-old who sang a...,-0.9739,0.0
3,2008-08-13,0.0,b' U.S. refuses Israel weapons to attack Iran:...,-0.9842,0.0
4,2008-08-14,0.0,b'All the experts admit that we should legalis...,-0.9774,0.0


In [14]:
#ACCURACY SCORE FOR VADER SENTIMENT ANLAYSIS
#We compare Vadersentiments score to VIX price if its up or down
vaderacc = accuracy_score(df['upordown'], df['Vadersentimentscore'])
vaderacc #60% accuracy

0.6018099547511312

# TextBlob

In [16]:
from textblob import TextBlob
# Get the polarity score using below function
def get_textBlob_score(sent):
    # This polarity score is between -1 to 1
    polarity = TextBlob(sent).sentiment.polarity
    return polarity

In [21]:
get_textBlob_score('great')

0.8